# Lab 1: Building a Q&A Bot with Iterative Prompt Refinement

## Objectives
- Build a Q&A system with context awareness
- Implement prompt refinement techniques
- Learn best practices for prompt engineering
- Understand conversation management

## Prerequisites
- Completed Module 1
- OpenAI API key configured

In [ ]:
import os
import json
import openai
from typing import List, Dict
from dotenv import load_dotenv

load_dotenv()
client = openai.OpenAI(
    api_key=os.getenv('OPENAI_API_KEY')
)

## 1. Basic Q&A System with Context Management

In [ ]:
class ConversationManager:
    def __init__(self, max_history: int = 5):
        self.conversation_history: List[Dict] = []
        self.max_history = max_history
    
    def add_message(self, role: str, content: str):
        self.conversation_history.append({"role": role, "content": content})
        if len(self.conversation_history) > self.max_history * 2:  # *2 because each exchange has 2 messages
            self.conversation_history = self.conversation_history[-self.max_history * 2:]
    
    def get_messages(self) -> List[Dict]:
        return self.conversation_history

class QABot:
    def __init__(self, system_prompt: str = None):
        self.conversation = ConversationManager()
        if system_prompt:
            self.conversation.add_message("system", system_prompt)
    
    def get_response(self, user_input: str) -> str:
        self.conversation.add_message("user", user_input)
        
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=self.conversation.get_messages(),
            temperature=0.7
        )
        
        assistant_response = response.choices[0].message.content
        self.conversation.add_message("assistant", assistant_response)
        
        return assistant_response

# Test the basic Q&A system
system_prompt = """You are a knowledgeable AI assistant specializing in technology and programming.
Always provide clear, concise answers with examples when appropriate."""

qa_bot = QABot(system_prompt)

questions = [
    "What is machine learning?",
    "Can you give me an example of supervised learning?"
]

for question in questions:
    print(f"User: {question}")
    response = qa_bot.get_response(question)
    print(f"Assistant: {response}\n")

## 2. Implementing Prompt Refinement

In [ ]:
class RefinedQABot(QABot):
    def __init__(self, system_prompt: str = None):
        super().__init__(system_prompt)
    
    def refine_question(self, user_input: str) -> str:
        refinement_prompt = f"""
        Please analyze and refine the following question to make it more specific and answerable:
        Question: {user_input}
        
        If the question is vague, add relevant context or specifications.
        If the question is complex, break it down into smaller parts.
        If the question is clear and specific, return it as is.
        
        Refined question:
        """
        
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": refinement_prompt}],
            temperature=0.3
        )
        
        return response.choices[0].message.content
    
    def get_response(self, user_input: str, refine: bool = True) -> str:
        if refine:
            refined_question = self.refine_question(user_input)
            print(f"Refined question: {refined_question}\n")
            return super().get_response(refined_question)
        return super().get_response(user_input)

# Test the refined Q&A system
refined_bot = RefinedQABot(system_prompt)

vague_questions = [
    "How do I start coding?",
    "What's the best way to learn AI?"
]

for question in vague_questions:
    print(f"User: {question}")
    response = refined_bot.get_response(question)
    print(f"Assistant: {response}\n")

## 3. Adding Domain-Specific Knowledge

In [ ]:
class ProgrammingQABot(RefinedQABot):
    def __init__(self):
        system_prompt = """
        You are an expert programming tutor specializing in Python and AI development.
        Follow these guidelines:
        1. Always include code examples when appropriate
        2. Explain concepts step by step
        3. Suggest best practices and common pitfalls
        4. Reference official documentation when relevant
        """
        super().__init__(system_prompt)
        
        self.code_templates = {
            "function": "def function_name(parameters):\n    """Docstring."""\n    # Function body\n    return result",
            "class": "class ClassName:\n    """Class docstring."""\n    def __init__(self):\n        # Initialize attributes\n        pass"
        }
    
    def format_code_response(self, code: str) -> str:
        return f"```python\n{code}\n```"
    
    def get_response(self, user_input: str) -> str:
        if "example" in user_input.lower() and "code" in user_input.lower():
            # Add code template to the conversation context
            template_prompt = f"Include relevant code template in your response. Here's a basic template: {self.format_code_response(self.code_templates['function'])}"
            self.conversation.add_message("system", template_prompt)
        
        return super().get_response(user_input)

# Test the programming-specific Q&A bot
programming_bot = ProgrammingQABot()

programming_questions = [
    "Can you show me an example of a Python function that calculates factorial?",
    "How do I create a simple class in Python?"
]

for question in programming_questions:
    print(f"User: {question}")
    response = programming_bot.get_response(question)
    print(f"Assistant: {response}\n")

## 4. Implementing Feedback Loop

In [ ]:
class FeedbackQABot(ProgrammingQABot):
    def __init__(self):
        super().__init__()
        self.feedback_history = []
    
    def incorporate_feedback(self, question: str, response: str, feedback: str) -> str:
        feedback_prompt = f"""
        Original question: {question}
        Previous response: {response}
        User feedback: {feedback}
        
        Please provide an improved response addressing the feedback.
        """
        
        improved_response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are an AI tutor focused on providing improved responses based on user feedback."},
                {"role": "user", "content": feedback_prompt}
            ],
            temperature=0.5
        )
        
        return improved_response.choices[0].message.content
    
    def get_response_with_feedback(self, question: str, feedback: str = None) -> str:
        if feedback and self.feedback_history:
            last_question, last_response = self.feedback_history[-1]
            improved_response = self.incorporate_feedback(last_question, last_response, feedback)
            self.feedback_history.append((question, improved_response))
            return improved_response
        
        response = self.get_response(question)
        self.feedback_history.append((question, response))
        return response

# Test the feedback system
feedback_bot = FeedbackQABot()

# Initial question
question = "How do I use a for loop in Python?"
print(f"User: {question}")
response = feedback_bot.get_response_with_feedback(question)
print(f"Assistant: {response}\n")

# Provide feedback and get improved response
feedback = "Could you provide more examples with different use cases?"
print(f"User feedback: {feedback}")
improved_response = feedback_bot.get_response_with_feedback(question, feedback)
print(f"Assistant (improved): {improved_response}")

## Exercises

1. Implement a specialized Q&A bot for a specific programming language or framework
2. Add support for code execution and validation in the responses
3. Implement a rating system for responses and use it to improve future answers
4. Create a system that can generate practice exercises based on the user's questions

## Next Steps
- Explore more advanced prompt engineering techniques
- Implement support for multiple programming languages
- Add code analysis and suggestions for improvement
- Integrate with development environments